In [1]:
from tokens import Tokens

In [2]:
import os
from qiskit_ibm_runtime import IBMRuntimeService
# Save an IBM Quantum account.
IBMRuntimeService.save_account(auth="legacy", token=Tokens.IBMQ_TOKEN, overwrite=True)
service = IBMRuntimeService(auth="legacy")

In [3]:
print(service.backends())

[<IBMBackend('ibmq_qasm_simulator')>, <IBMBackend('ibmq_armonk')>, <IBMBackend('ibmq_montreal')>, <IBMBackend('ibmq_toronto')>, <IBMBackend('ibmq_bogota')>, <IBMBackend('ibmq_mumbai')>, <IBMBackend('ibmq_lima')>, <IBMBackend('ibmq_belem')>, <IBMBackend('ibmq_quito')>, <IBMBackend('ibmq_guadalupe')>, <IBMBackend('ibmq_brooklyn')>, <IBMBackend('ibmq_jakarta')>, <IBMBackend('ibmq_manila')>, <IBMBackend('ibm_hanoi')>, <IBMBackend('ibm_lagos')>, <IBMBackend('ibm_cairo')>, <IBMBackend('ibm_auckland')>, <IBMBackend('ibm_perth')>, <IBMBackend('ibm_washington')>, <IBMBackend('ibmq_santiago')>, <IBMBackend('simulator_statevector')>, <IBMBackend('simulator_mps')>, <IBMBackend('simulator_extended_stabilizer')>, <IBMBackend('simulator_stabilizer')>]


In [160]:
hw_data = os.path.join(
    os.getcwd(), "../qiskit_machine_learning/runtime/ed_runtime_program.py"
)
hw_json = os.path.join(
    os.getcwd(), "eff_dim_metadata.json"
)

program_id = service.upload_program(data=hw_data, metadata=hw_json)
print(program_id)

eff-dim-A5Re9k41b8


In [161]:
service.pprint_programs()

eff-dim-A5Re9k41b8:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-OrQE3jYAWr:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-45jnA6Ndw5:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-05E6JqmEvy:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-pyY9Z69Ldy:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-45jXR2qALr:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-pyY4BNX7k5:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-XrgkJEB2qr:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-Wr09dBl2n8:
  Name: eff-dim
  Description: A sample runtime program.
eff-dim-2rGOMEdL4r:
  Name: eff-dim
  Description: A sample runtime program.
hello-world:
  Name: hello-world
  Description: A sample runtime program.
qaoa:
  Name: qaoa
  Description: Qiskit Runtime QAOA program
torch-train:
  Name: torch-train
  Description: A runtime program to train hybrid QNN models w

In [162]:
from qiskit_machine_learning.runtime.ed_runtime_client import EffDimRuntimeClient

In [163]:
from qiskit.circuit.library import ZFeatureMap, RealAmplitudes
from qiskit import Aer, QuantumCircuit
from qiskit_machine_learning.neural_networks import TwoLayerQNN, CircuitQNN

In [164]:
# create ranges for the number of data, n
# n = [5000, 8000, 10000, 40000, 60000, 100000, 150000, 200000, 500000, 1000000]
n=[[5000]]
# number of times to repeat the variational circuit
blocks = 1

# number of qubits, data samples and parameter sets to estimate the effective dimension
num_qubits = 3
num_inputs = 1
num_thetas = 1

# create a feature map
feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

# create a variational circuit
ansatz = RealAmplitudes(num_qubits, reps=blocks)

# create quantum circuit
qc = QuantumCircuit(num_qubits)
qc.append(feature_map, range(num_qubits))
qc.append(ansatz, range(num_qubits))
qc.decompose().draw( 'mpl')

# parity maps bitstrings to 0 or 1
def parity(x):
    return "{:b}".format(x).count("1") % 2
output_shape = 2  # corresponds to the number of classes, possible outcomes of the (parity) mapping.
# construct QNN
qnn = CircuitQNN(
    qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=parity,
    output_shape=output_shape,
    sparse=False
)

In [165]:
# backend = Aer.get_backend('qasm_simulator')
# backend = service.least_busy(operational=True, min_num_qubits=5)
backend = service.backend("simulator_statevector")
print(backend)
# token = "8d516ef8c3f646c312713365fa45a38b0d7ea5bd88f483a2be804511a375497d1631f6f2351ae06caef4c8962bbb91f7fcd07e779226f8449e485c735a5bc8bd"

<IBMBackend('simulator_statevector')>


In [166]:
# eff_dim = EffDimRuntimeClient(
#     service=service,
#     qnn=qnn,
#     mock_runtime=False,
#     backend=backend,
#     program_id = program_id
# )
#
# out = eff_dim.compute_eff_dim(n, num_inputs=num_inputs, num_thetas=num_thetas)

In [167]:
# print(out)

In [168]:
import dill, base64
from typing import List, Union, Callable, Optional, Any, Dict

def obj_to_str(obj: Any) -> str:
    """Encodes any object into a JSON-compatible string using dill. The intermediate
    binary data must be converted to base 64 to be able to decode into utf-8 format."""
    string = base64.b64encode(dill.dumps(obj, byref=False)).decode("utf-8")
    return string

def str_to_obj(string: str) -> Any:
    """Decodes a previously encoded string using dill (with an intermediate step
    converting the binary data from base 64)."""
    obj = dill.loads(base64.b64decode(string.encode()))
    return obj

In [169]:
import numpy as np
thetas = np.random.uniform(0, 1, size=(num_thetas, qnn.num_weights))
inputs = np.random.normal(0, 1, size=(num_inputs, qnn.num_inputs))

serial_inputs = inputs.tolist()
serial_thetas = thetas.tolist()
serial_qnn = obj_to_str(qnn)
# define runtime options
options = {"backend_name": backend.name}
inputs = {
            "qnn": serial_qnn,
            "inputs": serial_inputs,
            "thetas": serial_thetas,
            "shots": 1024,
            "measurement_error_mitigation": False,

            }

In [170]:
intermediate_info = {'args': []}

def callback(idx, args):
    print(args)
    intermediate_info['args'].append(args)

In [171]:
# send job to runtime and return result
import time
print("t0: ", time.time())
job = service.run(
    program_id=program_id,
    inputs=inputs,
    options=options,
    callback = callback
)

print(job.status())

result = job.result()
print(job.status())
print("t1: ", time.time())
# # print job ID if something goes wrong
# try:
#     result = job.result()
# except Exception as exc:
#     raise RuntimeError(f"The job {job.job_id()} failed unexpectedly.") from exc

t0:  1646751799.2500842
JobStatus.QUEUED
Beginning runtime program
qml_version0.3.1
Beginning runtime program
qml_version0.3.1
Grads: [[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]]
Outputs: [[0. 0.]]
Fwd: [[0.35380804 0.64619196]]
Grads: [[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]]
Outputs: [[0. 0.]]
Fwd: [[0.35380804 0.64619196]]
2022-03-08T15:03:32.296900897Z XXX lineno: 301, opcode: 129
2022-03-08T15:03:32.301848498Z Traceback (most recent call last):
2022-03-08T15:03:32.301848498Z   File "/code/program_starter.py", line 57, in <module>
2022-03-08T15:03:32.301848498Z     starter.run(user_params, main)
2022-03-08T15:03:32.301848498Z   File "/provider/programruntime/program_starter_wrapper.py", line 86, in run
2022-03-08T15:03:32.301848498Z     final_result = main(backend, messenger, **user_params)
2022-03-08T15:03:32.301848498Z   File "/code/program.py", line 305, in main
2022-03-08T15:03:32.301848498Z     result = ed.eff_dim(n)
2022-03-08T15:03:32.301848498Z   File "/code/program.py

RuntimeJobFailureError: 'Unable to retrieve job result. Job c8jn0dor7l5c7vv47ipg has failed:\n2022-03-08T15:03:32.296900897Z XXX lineno: 301, opcode: 129\n2022-03-08T15:03:32.301848498Z Traceback (most recent call last):\n2022-03-08T15:03:32.301848498Z   File "/code/program_starter.py", line 57, in <module>\n2022-03-08T15:03:32.301848498Z     starter.run(user_params, main)\n2022-03-08T15:03:32.301848498Z   File "/provider/programruntime/program_starter_wrapper.py", line 86, in run\n2022-03-08T15:03:32.301848498Z     final_result = main(backend, messenger, **user_params)\n2022-03-08T15:03:32.301848498Z   File "/code/program.py", line 305, in main\n2022-03-08T15:03:32.301848498Z     result = ed.eff_dim(n)\n2022-03-08T15:03:32.301848498Z   File "/code/program.py", line 170, in eff_dim\n2022-03-08T15:03:32.301848498Z     f_hat, trace = self.get_fhat()\n2022-03-08T15:03:32.301848498Z   File "/code/program.py", line 119, in get_fhat\n2022-03-08T15:03:32.301848498Z     grads, output = self.do

In [ ]:
print(job.status())
print(result)
job.cancel()
print(job.status())